In [ ]:
import time
import numpy as np
import pandas as pd

num_robot = 1
num_table = 9
num_product = 7
map_size = 20

In [ ]:
STATES = []

for a in range(map_size):
    for b in range(map_size):
        i = '0'+str(a) if a<10 else str(a)
        j = '0'+str(b) if b<10 else str(b)
        state = i+j
        STATES.append(state)

ACTIONS = ["left", "right",'up','down']

EPSILON = 0.9
ALPHA = 0.1
GAMMA = 0.9
FRESH_TIME = 0.3
TerminalFlag = "terminal"

CarryingMatrix = np.zeros((num_robot,num_product))
StoringMatrix = np.zeros((num_table,num_product))

table_pos = [
    [1,1],[3,3],[5,5],[7,7],[9,9],[11,11],[13,13],[15,15],[17,17]
]

In [ ]:
def build_q_table(states, actions):
    return pd.DataFrame(
        np.zeros((len(states), len(actions))),
        index = states,
        columns = actions
    )


def choose_action(S, q_table):
    S0 = S[0]+S[1]+S[2]+S[3]
    state_table = q_table.loc[S0, :]
    if (np.random.uniform() > EPSILON) or ((state_table == 0).all()):
        action_name = np.random.choice(ACTIONS)
    else:
        action_name = state_table.idxmax()
    return action_name


In [ ]:
def get_env_feedback(S, A):
    if S[0]+S[1] == '00' and A == 'up' or S[0]+S[1] == str(map_size-1) and A == 'down' or S[2]+S[3] == '00' and A == 'left' or S[2]+S[3] == str(map_size-1) and A == 'right':
        R = -100000
        S_ = S
    else :
        R = 0
        x = int(S[0]+S[1])
        y = int(S[2]+S[3])
        #print(x,y)
        if x == table_pos[0][0] and y == table_pos[0][1]:
            if CarryingMatrix[0][0] == 0 :
                CarryingMatrix[0][0] = 1
                R = -30
        elif x == table_pos[1][0] and y == table_pos[1][1]:
            if CarryingMatrix[0][1] == 0 :
                CarryingMatrix[0][1] = 1
                R = -44
        elif x == table_pos[2][0] and y == table_pos[2][1]:
            if CarryingMatrix[0][2] == 0 :
                CarryingMatrix[0][2] = 1
                R = -58
        elif x == table_pos[3][0] and y == table_pos[3][1]:
            if CarryingMatrix[0][3] == 0 and StoringMatrix[3][0] == 1 and StoringMatrix[3][1] == 1:
                CarryingMatrix[0][3] = 1
                StoringMatrix[3][0] == 0
                StoringMatrix[3][1] == 0
                R = -154
            if CarryingMatrix[0][0] == 1 :
                CarryingMatrix[0][0] = 0
                StoringMatrix[3][0] = 1
                R = 60
            if CarryingMatrix[0][1] == 1 :
                CarryingMatrix[0][1] = 0
                StoringMatrix[3][1] =1
                R = 76
        elif x == table_pos[4][0] and y == table_pos[4][1]:
            if CarryingMatrix[0][4] == 0 and StoringMatrix[4][0] == 1 and StoringMatrix[4][2] == 1:
                CarryingMatrix[0][4] = 1
                StoringMatrix[4][0] == 0
                StoringMatrix[4][2] == 0
                R = -172
            if CarryingMatrix[0][0] == 1 :
                CarryingMatrix[0][0] = 0
                StoringMatrix[4][0] =1
                R = 60
            if CarryingMatrix[0][2] == 1 :
                CarryingMatrix[0][2] = 0
                StoringMatrix[4][2] =1
                R = 92
        elif x == table_pos[5][0] and y == table_pos[5][1]:
            if CarryingMatrix[0][5] == 0 and StoringMatrix[5][1] == 1 and StoringMatrix[5][2] == 1:
                CarryingMatrix[0][5] = 1
                StoringMatrix[5][1] == 0
                StoringMatrix[5][2] == 0
                R = -192
            if CarryingMatrix[0][1] == 1 :
                CarryingMatrix[0][1] = 0
                StoringMatrix[5][1] = 1
                R = 76
            if CarryingMatrix[0][2] == 1 :
                CarryingMatrix[0][2] = 0 
                StoringMatrix[5][2] == 1  
                R = 92 
        elif x == table_pos[6][0] and y == table_pos[6][1]:
            if CarryingMatrix[0][6] == 0 and StoringMatrix[6][3] == 1 and StoringMatrix[6][4] == 1 and StoringMatrix[6][5] == 1:
                CarryingMatrix[0][6] = 1
                StoringMatrix[6][3] = 0
                StoringMatrix[6][4] = 0
                StoringMatrix[6][5] = 0
                R = -760
            if CarryingMatrix[0][3] == 1 :
                CarryingMatrix[0][3] = 0
                StoringMatrix[6][3] = 1
                R = 225
            if CarryingMatrix[0][4] == 1 :
                CarryingMatrix[0][4] = 0  
                StoringMatrix[6][4] = 1
                R = 250
            if CarryingMatrix[0][5] == 1 :
                CarryingMatrix[0][5] = 0  
                StoringMatrix[6][5] = 1
                R = 275
        elif x == table_pos[7][0] and y == table_pos[7][1]:
            if CarryingMatrix[0][6] == 1 :
                CarryingMatrix[0][6] = 0
                R = 1050
        elif x == table_pos[8][0] and y == table_pos[8][1]:
            if CarryingMatrix[0][0] == 1 :
                CarryingMatrix[0][0] = 0
                R = 60
            if CarryingMatrix[0][1] == 1 :
                CarryingMatrix[0][1] = 0
                R = 76
            if CarryingMatrix[0][2] == 1 :
                CarryingMatrix[0][2] = 0
                R = 92
            if CarryingMatrix[0][3] == 1 :
                CarryingMatrix[0][3] = 0
                R = 225
            if CarryingMatrix[0][4] == 1 :
                CarryingMatrix[0][4] = 0
                R = 250
            if CarryingMatrix[0][5] == 1 :
                CarryingMatrix[0][5] = 0
                R = 275  
            if CarryingMatrix[0][6] == 1 :
                CarryingMatrix[0][6] = 0
                R = 1050
          
        R = R + 1000 if R != 0 else 0 
        if A == 'up':
            x = x-1
            x = '0'+str(x) if x<10 else str(x)
            S[0] = x[0]
            S[1] = x[1]
            S_ = S
        elif A == 'down':
            x = x+1
            x = '0'+str(x) if x<10 else str(x)
            S[0] = x[0]
            S[1] = x[1]
            S_ = S
        elif A == 'left':
            y = y-1
            y = '0'+str(y) if y<10 else str(y)
            S[2] = y[0]
            S[3] = y[1]
            S_ = S
        elif A == 'right':
            y = y+1
            y = '0'+str(y) if y<10 else str(y)
            S[2] = y[0]
            S[3] = y[1]
            S_ = S
    return S_, R

In [ ]:
def update_env(S, episode, step_counter):
    env_list = ["-"] * (len(STATES) - 1) + ["T"]
    if S == TerminalFlag:
        interaction = 'Episode %s: total_steps = %s' % (episode + 1, step_counter)
        print(interaction)
        time.sleep(2)
    else:
        env_list[S] = '0'
        interaction = ''.join(env_list)
        print(interaction)
        time.sleep(FRESH_TIME)


In [ ]:
MAX_EPISODES = 2
def rl():
    q_table = build_q_table(STATES, ACTIONS)
    q_table = Q_saved
    for episode in range(MAX_EPISODES):
        step_counter = 0
        S = ['0','0','0','0']
        is_terminated = False
        #update_env(S, episode, step_counter)
        while not is_terminated:
            A = choose_action(S, q_table)
            S_, R = get_env_feedback(S, A)
            S0 = S[0]+S[1]+S[2]+S[3]
            S1 = S_[0]+S_[1]+S_[2]+S_[3]
            q_predict = q_table.loc[S0, A]

            if S_ != TerminalFlag:
                q_target = R + GAMMA * q_table.loc[S1, :].max()
            else:
                q_target = R
                is_terminated = True
            q_table.loc[S0, A] += ALPHA * (q_target - q_predict)
            S = S_
            #update_env(S, episode, step_counter + 1)
            step_counter += 1
            if step_counter % 10000 == 0:
                print('epoch%d,step%d'%(episode,step_counter))
            if step_counter > 999999 :
                break
    return q_table

q_table = rl()
print(q_table)


epoch0,step10000
epoch0,step20000
epoch0,step30000
epoch0,step40000
epoch0,step50000
epoch0,step60000
epoch0,step70000
epoch0,step80000
epoch0,step90000
epoch0,step100000
epoch0,step110000
epoch0,step120000
epoch0,step130000
epoch0,step140000
epoch0,step150000
epoch0,step160000
epoch0,step170000
epoch0,step180000
epoch0,step190000
epoch0,step200000
epoch0,step210000
epoch0,step220000
epoch0,step230000
epoch0,step240000
epoch0,step250000
epoch0,step260000
epoch0,step270000
epoch0,step280000
epoch0,step290000
epoch0,step300000
epoch0,step310000
epoch0,step320000
epoch0,step330000
epoch0,step340000
epoch0,step350000
epoch0,step360000
epoch0,step370000
epoch0,step380000
epoch0,step390000
epoch0,step400000
epoch0,step410000
epoch0,step420000
epoch0,step430000
epoch0,step440000
epoch0,step450000
epoch0,step460000
epoch0,step470000
epoch0,step480000
epoch0,step490000
epoch0,step500000
epoch0,step510000
epoch0,step520000
epoch0,step530000
epoch0,step540000
epoch0,step550000
epoch0,step560000
e

In [ ]:
Q_saved = q_table
np.save('./Qsave',Q_saved)
Q_saved

,left,right,up,down
0000,-0.104982,0.0,-99105.966821,0.000000
0001,0.000000,0.0,-72266.770419,0.000000
0002,0.000000,0.0,-59548.215554,0.000000
0003,0.000000,0.0,-304.944844,0.000000
0004,0.000000,0.0,-20656.863851,0.000000
...,...,...,...,...
1915,0.000000,0.0,0.000000,-10574.263969
1916,0.000000,0.0,0.000000,-12745.133143
1917,0.000000,0.0,0.000000,-9849.704052
1918,0.000000,0.0,0.000000,-10333.682223
